<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/DEIT_Diagnosis_Predict" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEIT Training

This notebook is used to train DEIT model for glacoma diagnosis.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
import torch


In [ ]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = False #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = 'cache'        # Directory in which to cache materialized BDBags for datasets
working_dir = 'working'    # Directory in which to place output files for later upload.

configuration_rid="2-A5JP"  # 10% of subjects for train, validation, and test set
# configuration_rid="2-A5JR"   Full set of subjects for train, validation, and test set

In [ ]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0]
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir), 
                                                                       output_dir = working_dir, 
                                                                       crop_to_eye=False)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir), 
                                                                                 output_dir = working_dir, 
                                                                                 crop_to_eye=False)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir), 
                                                                     output_dir = working_dir, 
                                                                     crop_to_eye=False)


In [ ]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai.models.deit_diagnosis_predict import predict
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  model = torch.load(PATH)

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)